In [1]:
import os 
import numpy as np



## Necessary paths


In [2]:
#main folder
root_path = ""

#folder to store bounding boxes and frames
path_bounding_boxes_frames = ""
#folder to store dat and npy files
path_dat_npy_files =   ""




## Finding .dat files in our folder

In [28]:

#Changing to dat_npy_files directory
os.chdir(path_dat_npy_files)

#We write txt file with the paths of the dat.files we used in root folder
used_dat_files_txt = open(f"{root_path}" + "/used_dat_files.txt","w")



for current_dir, dirs, files in os.walk('.'):
    # Going through all files
    for f in files:
        # Checking if filename ends with '.dat'
        if f.endswith('dat'):
            #Saving dat file each to the txt fil
            used_dat_files_txt.write(path_dat_npy_files + f"/{f}\n")
            
used_dat_files_txt.close()

## Data visualizer, frame and bounding boxes extractor

In [29]:


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import numpy as np
import cv2
import argparse
from glob import glob

from src.visualize import vis_utils as vis

from src.io.psee_loader import PSEELoader





def play_files_parallel(td_files, labels=None, delta_t=20000, skip=0,count_dat_file = 0):
    """
    Plays simultaneously files and their boxes in a rectangular format.
    """
    # open the video object for the inSput files
    videos = [PSEELoader(td_file) for td_file in td_files]
    # use the naming pattern to find the corresponding box file
    box_videos = [PSEELoader(glob(td_file.split('_td.dat')[0] +  '*.npy')[0]) for td_file in td_files]

    height, width = videos[0].get_size()
    print("Hola",height,width)
    labelmap = vis.LABELMAP if height == 240 else vis.LABELMAP_LARGE

    # optionally skip n minutes in all videos
    for v in videos + box_videos:
        v.seek_time(skip)

    # preallocate a grid to display the images
    size_x = int(math.ceil(math.sqrt(len(videos))))
    size_y = int(math.ceil(len(videos) / size_x))
    frame = np.zeros((size_y * height, width * size_x, 3), dtype=np.uint8)

    cv2.namedWindow('out', cv2.WINDOW_NORMAL)
    
    count = 0 

    # while all videos have something to read
    while not sum([video.done for video in videos]):

        # load events and boxes from all files
        events = [video.load_delta_t(delta_t) for video in videos]
        box_events = [box_video.load_delta_t(delta_t) for box_video in box_videos]
        
        #Save boxes of each frame to a text file
        bboxes_file = open(path_bounding_boxes_frames + f"/prophesee_image_{count_dat_file}_{count}.txt","w")
            
        for box in box_events:
            bboxes_file.write(f"{box}\n")
        
        bboxes_file.close()
            
           
        for index, (evs, boxes) in enumerate(zip(events, box_events)):
            y, x = divmod(index, size_x)
            # put the visualization at the right spot in the grid
            im = frame[y * height:(y + 1) * height, x * width: (x + 1) * width]
            # call the visualization functions
            im = vis.make_binary_histo(evs, img=im, width=width, height=height)

            #vis.draw_bboxes(im, boxes, labelmap=labelmap)
            
           

        # display the result
        
        cv2.imshow('out', frame)
        cv2.imwrite(path_bounding_boxes_frames + f'/image_{count_dat_file}_{count}.jpg',frame)
        cv2.waitKey(1)
        
        count += 1




## Extracting frames and bounding boxes for each .dat file 

In [ ]:
#Back to main path, just in case
os.chdir(root_path)

count_dat_file = 0

#Reading used_dat_files.txt first
used_dat_files = open(f"{root_path}" + "/used_dat_files.txt","r+")

dat_files = used_dat_files. readlines()
#We have to pass the .dat paths as a list
#We apply the extractor to each file:

paths = []
for dat_file in dat_files:
    paths.append(dat_file[:-1])
    
for i in range(len(paths)):
        try:
            
           play_files_parallel([paths[i]], labels=None, delta_t=50000, skip=0,count_dat_file=count_dat_file)
        except:
           pass
        count_dat_file += 1 

